# Pandas Test - Real life exercise

In [59]:
import pandas as pd
import itertools

For visualizations:

In [90]:
import cufflinks as cf; cf.go_offline()
import plotly_express as px

In [30]:
ad_recs_annotated = pd.read_csv('data/ad_annotations.csv')
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider
284462,2022-week_01,AD08G88NPJ,Extreme DA,US,yes,NaN,Yield,RNNY
294531,2022-week_02,AD08LZZ87M,Puentes Company,IN,yes,NaN,Yield,UserPopY
278152,2022-week_01,AD08PG68WL,MoJoe Brewing Company,IN,no,True,Yield,UserPopY
306199,2022-week_03,AD078CWTPG,MoJoe Brewing Company,IN,no,True,Yield,UserPopY
282827,2022-week_01,AD08P86L8K,Extreme DA,US,yes,NaN,Yield,RNNY


### Data Modifications

In [128]:
ad_recs_annotated['week_number'] = ad_recs_annotated.week_id.apply(lambda x: int(x.split('_')[1]))

#### Q1 - Add a column with the sample year

In [53]:
ad_recs_annotated['year'] = ad_recs_annotated.week_id.apply(lambda row: str(row)[:4]) # first question

#### Q2 - Add a column with the sample quarter (google how to)

In [33]:
ad_recs_annotated['quarter'] = ad_recs_annotated.week_id.apply(lambda w: int(w.split('_')[1]) // 13 + 1)

#### Q3 - Add a new success column with values 1 and 0 for yes and no 

In [34]:
ad_recs_annotated['new_success_col'] = ad_recs_annotated.is_success.apply(lambda row: 1 if row=='yes' else 0) # third question

#### Q4

Are there any NaNs in the is_sever column? Count how many and create a new column with NaNs filled as False. Check that indeed the new column contaion no NaNs.

In [35]:
ad_recs_annotated.is_sever.isnull().sum() # Yes, we have NaNs

304195

In [36]:
ad_recs_annotated['is_severe_fixed'] = ad_recs_annotated.is_sever.fillna(False)

In [37]:
ad_recs_annotated['is_severe_fixed'].isnull().sum()

0

#### Q5 - Capitlize (first letter only) the is_success columnsum

In [38]:
ad_recs_annotated['is_success_capital'] = ad_recs_annotated['is_success'].str.capitalize()
ad_recs_annotated.is_success_capital.head()

0    Yes
1    Yes
2    Yes
3    Yes
4    Yes
Name: is_success_capital, dtype: object

* Are there duplicated ads? To compac the data, remove duplicated ads and instead add an ad_count column **(pay attention, this is not trivial at all)**

In [ ]:
ad_counts = ad_recs_annotated.groupby(['ad_id', 'week_id']).size().reset_index().rename(columns={0:'count'})
ad_counts.sort_values(by='count', ascending=False)[:5]
def count_to_ad(row):
    return ad_counts.query(f'ad_id == "{row.ad_id}" and week_id == "{row.week_id}"')['count']
# count_to_ad()
sample_to_test = ad_recs_annotated.sample(2).loc[:,['ad_id','week_id']]
ad_count_col = sample_to_test.loc[:,['ad_id','week_id']].apply(count_to_ad,axis=1)
ad_count_col.head(1)
# ad_counts
# sample_to_test

### Subset selection

#### Q1 - How many requests are there in US region?

In [39]:
ad_recs_annotated[ad_recs_annotated.region=='US'].shape[0] # 1st question

103846

#### Q2 - How many successful requests are there in US region?

In [40]:
ad_recs_annotated.query('region == "US" and is_success=="yes"').shape[0] # 2nd question

88918

#### Q3 - How many successful requests are there in US region, on week 52?

In [41]:
ad_recs_annotated.query('region == "US" and is_success=="yes" and week_id.str.endswith("52")', engine='python').shape[0] # 3rd question

3342

#### Q4 - How many requests where delivered by DNNQ, RNNY and UserPopQ together?

In [42]:
ad_recs_annotated.query('rec_provider == "DNNQ" or rec_provider == "RNNY" or rec_provider == "UserPopQ"').shape[0] # 4th question

200093

#### Q5 - How many requests where delivered by rule based providers?

In [43]:
ad_recs_annotated.rec_provider.value_counts()['RuleBased'] # 5th question

182

#### Q6 - Select only the requests from 2021 - How many are there?

In [44]:
ad_recs_annotated.query('sample_year == "2021" and is_success=="yes"').shape[0] # 6th question

221145

#### Q7 - Select the reqeusts from week 30 to week 40 - How many are there?

In [45]:
def is_between_week_30_t0_40(week_id):
    week_num=[]
    for i in range(30,41,1):
        if week_id.endswith(str(i)):
            return True
    return False
ad_recs_annotated[ad_recs_annotated.week_id.apply(is_between_week_30_t0_40)].shape[0] #7th question

115051

#### Q8 - Select all the data that comes from the most popular Ad

In [46]:
most_popular_ad = ad_recs_annotated.groupby(['ad_id']).size().idxmax() #8th question A
ad_recs_annotated[ad_recs_annotated.ad_id==most_popular_ad] #8th question B

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,sample_year,quarter,new_success_col,is_severe_fixed,is_success_capital
7031,2021-week_33,AD07PFFMP9,Puentes Company,IN,yes,NaN,Yield,ManualQ,2021,3,1,False,Yes
7032,2021-week_33,AD07PFFMP9,Puentes Company,IN,yes,NaN,Yield,ManualQ,2021,3,1,False,Yes
20010,2021-week_34,AD07PFFMP9,Bizanga,IN,yes,NaN,Yield,ManualQ,2021,3,1,False,Yes
20011,2021-week_34,AD07PFFMP9,Puentes Company,IN,yes,NaN,Yield,ManualQ,2021,3,1,False,Yes
35214,2021-week_35,AD07PFFMP9,Bizanga,IN,yes,NaN,Yield,ManualQ,2021,3,1,False,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363533,2022-week_52,AD07PFFMP9,MoJoe Brewing Company,IN,yes,NaN,Yield,ManualQ,2022,5,1,False,Yes
363534,2022-week_52,AD07PFFMP9,MoJoe Brewing Company,IN,yes,NaN,Yield,ManualQ,2022,5,1,False,Yes
363535,2022-week_52,AD07PFFMP9,Bizanga,IN,yes,NaN,Yield,ManualQ,2022,5,1,False,Yes
363536,2022-week_52,AD07PFFMP9,MoJoe Brewing Company,IN,yes,NaN,Yield,ManualQ,2022,5,1,False,Yes


#### Q9 - Select all the data that comes from the least popular provider

In [47]:
least_popular_ad = ad_recs_annotated.groupby(['ad_id']).size().idxmin() #9th question A
ad_recs_annotated[ad_recs_annotated.ad_id==least_popular_ad] #9th question B

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,sample_year,quarter,new_success_col,is_severe_fixed,is_success_capital
70085,2021-week_37,AD00000088,Extreme DA,US,yes,NaN,Yield,BooksY,2021,3,1,False,Yes


#### Q10 - Select the data in which is_sever is None. How big is it?

In [48]:
ad_recs_annotated.is_sever.value_counts(dropna=False)[0] #10th question

304195

#### Q11 - Select the data in which is_sever is None. How big is it?

In [49]:
def is_req_3_words(requester): #11th question A
    return len(str(requester).split(' ')) == 3 #11th question B
ad_recs_annotated[ad_recs_annotated.requester.apply(is_req_3_words)] #11th question C

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,sample_year,quarter,new_success_col,is_severe_fixed,is_success_capital
0,2021-week_33,AD0088VOS,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False,Yes
1,2021-week_33,AD07KYS8JM,MoJoe Brewing Company,DE,yes,NaN,Yield,DNNY,2021,3,1,False,Yes
2,2021-week_33,AD08PDP6Y9,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False,Yes
3,2021-week_33,AD89608808,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False,Yes
4,2021-week_33,AD07CMVHP6,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
373663,2022-week_52,AD09G98VJZ,MoJoe Brewing Company,US,yes,NaN,Yield,RNNY,2022,5,1,False,Yes
373689,2022-week_52,AD078WDG8W,MoJoe Brewing Company,US,yes,NaN,Yield,RNNY,2022,5,1,False,Yes
373691,2022-week_52,AD08M86WW8,MoJoe Brewing Company,US,yes,NaN,Yield,DNNY,2022,5,1,False,Yes
373722,2022-week_52,AD09C8X6VY,Search Million Culture,US,yes,NaN,Qality,DNNQ,2022,5,1,False,Yes


#### Q12 - Select the data in which the requester is a 3 word company, and the ad_id has the letter 6 in it

In [50]:
three_word_reqs = ad_recs_annotated[ad_recs_annotated.requester.apply(is_req_3_words)] #12th question A
three_word_reqs[three_word_reqs.ad_id.str.contains('6')] #12th question B

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,sample_year,quarter,new_success_col,is_severe_fixed,is_success_capital
2,2021-week_33,AD08PDP6Y9,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False,Yes
3,2021-week_33,AD89608808,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False,Yes
4,2021-week_33,AD07CMVHP6,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False,Yes
11,2021-week_33,AD88967068,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False,Yes
41,2021-week_33,AD07L6JC8W,MoJoe Brewing Company,DE,yes,NaN,Yield,DNNY,2021,3,1,False,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
373364,2022-week_52,AD06XJR86M,MoJoe Brewing Company,US,yes,NaN,Yield,RuleBasedY,2022,5,1,False,Yes
373394,2022-week_52,AD08Y6R9WV,MoJoe Brewing Company,US,yes,NaN,Yield,DNNY,2022,5,1,False,Yes
373511,2022-week_52,AD09C68T8Y,MoJoe Brewing Company,US,yes,NaN,Yield,DNNY,2022,5,1,False,Yes
373691,2022-week_52,AD08M86WW8,MoJoe Brewing Company,US,yes,NaN,Yield,DNNY,2022,5,1,False,Yes


#### Q13 - Select the data in which the requester is a 3 word company, and the multiplication of all the numbers in the ad_id is bigger than 30

In [51]:
def is_mult_over_30(ad_id): #13th question A
    tot = 1
    for i in ad_id:
        if i in ['0','1','2','3','4','5','6','7','8','9']:
            tot *= int(i)
    return tot > 30 #13th question B
ad_recs_annotated[ad_recs_annotated.ad_id.apply(is_mult_over_30)] #13th question C

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,sample_year,quarter,new_success_col,is_severe_fixed,is_success_capital
37,2021-week_33,AD88898888,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False,Yes
76,2021-week_33,AD88898786,MoJoe Brewing Company,DE,yes,NaN,Yield,BooksY,2021,3,1,False,Yes
112,2021-week_33,AD88788887,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False,Yes
162,2021-week_33,AD88887688,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False,Yes
174,2021-week_33,AD88888876,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372642,2022-week_52,AD86888898,Extreme DA,US,yes,NaN,Yield,BooksY,2022,5,1,False,Yes
372772,2022-week_52,AD88688788,Extreme DA,US,yes,NaN,Yield,BooksY,2022,5,1,False,Yes
372782,2022-week_52,AD88888877,Extreme DA,US,yes,NaN,Yield,BooksY,2022,5,1,False,Yes
373219,2022-week_52,AD88998888,Extreme DA,US,yes,NaN,Yield,BooksY,2022,5,1,False,Yes


### Analysis

#### Q1 - How many weeks available in the data? Are there any missing weeks?

In [56]:
ad_recs_annotated.groupby(['year','week_number']).week_id.nunique() #1st question - week 52 of 2021 missing.

year  week_number
2021  33             1
      34             1
      35             1
      36             1
      37             1
      38             1
      39             1
      40             1
      41             1
      42             1
      43             1
      44             1
      45             1
      46             1
      47             1
      48             1
      49             1
      50             1
      51             1
2022  1              1
      2              1
      3              1
      4              1
      5              1
      6              1
      52             1
Name: week_id, dtype: int64

#### Q2 - How many regions available in the data? Are there any missing weeks per region?

In [62]:
def find_missing_week_id(week_ids):
    missing_weeks = []
    for i in range (33, 53):
        if i not in week_ids:
            missing_weeks.append(i)
    return missing_weeks
ad_recs_annotated.groupby('region').week_number.unique().apply(find_missing_week_id)

region
DE      []
GB      []
IN      []
JP    [51]
US      []
Name: week_number, dtype: object

#### Q3 - How many ads are being sent to annotation in each region per week?

In [63]:
ad_recs_annotated.groupby(['region','week_id']).ad_id.nunique().unstack('region') # 3rd question

region,DE,GB,IN,JP,US
week_id,,,,,
2021-week_33,2346.0,3062.0,3023.0,1561.0,3832.0
2021-week_34,2315.0,3090.0,3057.0,1551.0,3909.0
2021-week_35,2324.0,3073.0,3056.0,1505.0,3888.0
2021-week_36,2319.0,3070.0,3030.0,1548.0,3868.0
2021-week_37,2325.0,3036.0,3040.0,1591.0,3868.0
2021-week_38,2334.0,3049.0,3011.0,1516.0,3899.0
2021-week_39,2323.0,2996.0,2958.0,1510.0,3920.0
2021-week_40,2325.0,2958.0,2592.0,1553.0,3908.0
2021-week_41,2271.0,2984.0,2859.0,1519.0,3901.0


#### Q4 - How many None values are there in is_sever column?

In [64]:
ad_recs_annotated[(ad_recs_annotated.is_sever.isnull())].shape[0] # 4th question

304195

#### Q5 - Are ads being sent more than one time in a given week? what is the ad_id that was sent the most in a given week? (e.g. ad X where sent 50 times in week Y)

In [66]:
tmp_data = ad_recs_annotated.groupby(['ad_id','week_id']).size().unstack('week_id').idxmax().to_frame().reset_index().rename(columns={0:"most_popular_ad_id"}) # 5th question
most_popular_ad_id = ad_recs_annotated.ad_id.value_counts().idxmax() # most popular ad
most_popular_ad_id

'AD07PFFMP9'

#### Q6

6. {Are there ads that are being sent in more than one week?}
   1. Which is the ad that was sent in most amount of weeks (e.g. ad X where sent in Y different weeks)
   2. What are the weeks that the ad from above was sent in? 
   3. Is there an Ad that was succsefull in one week, but not succesfful in a different week? Show one.

In [69]:
ad_recs_annotated.groupby(['ad_id']).nunique().sort_values(by='week_id', ascending=False).iloc[0] # 6th.A question

week_id                26
requester               9
region                  1
is_success              1
is_sever                0
recommendation_type     2
rec_provider            1
sample_year             2
quarter                 4
new_success_col         1
is_severe_fixed         1
is_success_capital      1
year                    2
week_number            26
Name: AD07PFFMP9, dtype: int64

In [70]:
ad_recs_annotated[ad_recs_annotated.ad_id==ad_recs_annotated.groupby(['ad_id']).nunique().sort_values(by='week_id', ascending=False).iloc[0].name].week_id.unique() #6th.B question

array(['2021-week_33', '2021-week_34', '2021-week_35', '2021-week_36',
       '2021-week_37', '2021-week_38', '2021-week_39', '2021-week_40',
       '2021-week_41', '2021-week_42', '2021-week_43', '2021-week_44',
       '2021-week_45', '2021-week_46', '2021-week_47', '2021-week_48',
       '2021-week_49', '2021-week_50', '2021-week_51', '2022-week_01',
       '2022-week_02', '2022-week_03', '2022-week_04', '2022-week_05',
       '2022-week_06', '2022-week_52'], dtype=object)

In [71]:
temp=ad_recs_annotated.groupby(['week_id','ad_id','is_success']).size().unstack(['is_success'])# 6th.C question 
temp = temp.dropna()# 6th.C question 
temp.iloc[:50]# 6th.C question 
ad_recs_annotated[ad_recs_annotated.ad_id=='AD097GD8GP'].head(5) # 6th.C question example

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,sample_year,quarter,new_success_col,is_severe_fixed,is_success_capital,year,week_number
7323,2021-week_33,AD097GD8GP,Search Million Culture,IN,yes,NaN,Qality,UserPopQ,2021,3,1,False,Yes,2021,33
7928,2021-week_33,AD097GD8GP,Puentes Company,IN,no,True,Yield,UserPopY,2021,3,0,True,No,2021,33
20412,2021-week_34,AD097GD8GP,Puentes Company,IN,no,True,Yield,UserPopY,2021,3,0,True,No,2021,34
21178,2021-week_34,AD097GD8GP,Search Million Culture,IN,yes,NaN,Qality,UserPopQ,2021,3,1,False,Yes,2021,34
93423,2021-week_39,AD097GD8GP,Puentes Company,IN,no,True,Yield,UserPopY,2021,4,0,True,No,2021,39


#### Q7 - When is_sever is None, what is the number of successful requests? What is the number of non-successful requests? What do you learn from it about the reason for Nones in the first place?

In [ ]:
# ad_recs_annotated[ad_recs_annotated.is_sever == True].is_success.value_counts() #7th question - only 12 'yes', probably typos
# ad_recs_annotated.groupby(['region','recommendation_type']).size().apply(lambda row: row / ad_recs_annotated.groupby('region').size().sum())
# temp_df = ad_recs_annotated.groupby(['region','recommendation_type']).size().to_frame().rename(columns = {0:'count'}).unstack('recommendation_type')
# temp_df

#### Q8 - Per each region, What is the Quality/Yield traffic proportion WoW?

In [72]:
a = ad_recs_annotated.groupby(['region','recommendation_type']).size() #8th question
b = ad_recs_annotated.groupby('region').size() #8th question
a.div(b) #8th question

region  recommendation_type
DE      Qality                 0.808900
        Yield                  0.191100
GB      Qality                 0.836342
        Yield                  0.163658
IN      Qality                 0.491281
        Yield                  0.508719
JP      Qality                 0.852067
        Yield                  0.147933
US      Qality                 0.390511
        Yield                  0.609489
dtype: float64

#### Q9 - How many different reqeusters are there?

In [74]:
ad_recs_annotated.requester.nunique() #9th question

36

#### Q10 - Which are the top 5 requesters per region?

In [76]:
ad_recs_annotated.groupby('region').apply(lambda grp: grp.requester.value_counts().head(5)).to_frame() # 10th question

requester
region                                  
DE     Search Million Culture      23657
       RelayFoods                  17313
       MoJoe Brewing Company        8616
       iDreamsky Technology         3542
       SOLOMO365                    2435
GB     Search Million Culture      28002
       RelayFoods                  15920
       Fancy                       13678
       MoJoe Brewing Company        9530
       iDreamsky Technology         3919
IN     Puentes Company             29427
       Search Million Culture      21365
       RelayFoods                  17352
       MoJoe Brewing Company        7191
       iDreamsky Technology         1995
JP     RelayFoods                  24574
       Search Million Culture       7906
       MoJoe Brewing Company        3549
       LocalVox Media               1724
       Tab Solutions                1366
US     Extreme DA                  57215
       Fancy                       18668
       Tab Solutions                9790
       Search Million Culture       5610
       Sensor Tower                 3503

#### Q11 - Which are the different rec_proivders?

In [77]:
ad_recs_annotated.rec_provider.unique() # 11th question

array(['DNNQ', 'DNNY', 'BooksQ', 'UserPopQ', 'UserPopY', 'RuleBasedY',
       'ManualY', 'ManualQ', 'BooksY', 'XGBQ', 'XGBY', 'RNNQ', 'RNNY',
       'RuleBased', 'UserPopSelectionQ', 'UserPopSelectionY'],
      dtype=object)

#### Q12 - Are there different rec providers per region?

In [78]:
ad_recs_annotated.groupby('region').apply(lambda grp: grp.rec_provider.unique()) # 12th question, answer's yes.

region
DE    [DNNQ, DNNY, BooksQ, UserPopQ, UserPopY, RuleB...
GB    [DNNY, XGBQ, DNNQ, XGBY, RNNQ, UserPopQ, Manua...
IN    [UserPopY, RuleBasedY, UserPopQ, ManualY, Manu...
JP    [ManualQ, UserPopQ, RuleBasedY, DNNQ, ManualY,...
US    [DNNY, ManualQ, DNNQ, UserPopY, XGBY, RNNY, Ma...
dtype: object

#### Q13 - Are there different rec providers per rec type?

In [79]:
ad_recs_annotated.groupby('recommendation_type').apply(lambda grp: grp.rec_provider.unique()) # 13th question, answer's yes.

recommendation_type
Qality    [DNNQ, BooksQ, UserPopQ, ManualY, ManualQ, XGB...
Yield     [DNNY, UserPopY, RuleBasedY, BooksY, ManualY, ...
dtype: object

#### Q14 - What are the notation rules distinguishing between quality vs yielding providers? (quality providers end in 'q'?)

In [80]:
ad_recs_annotated.groupby('recommendation_type').rec_provider.unique() # 14th question. generally end in Q if quality, or Y if yield.

recommendation_type
Qality    [DNNQ, BooksQ, UserPopQ, ManualY, ManualQ, XGB...
Yield     [DNNY, UserPopY, RuleBasedY, BooksY, ManualY, ...
Name: rec_provider, dtype: object

#### Q15 - Which is the most successful region of operation?

In [81]:
ad_recs_annotated.groupby('is_success').region.value_counts()['yes'].head(1) # 15th question

region
US    88918
Name: region, dtype: int64

#### Q16 - Which is the most successful rec provider per each region?

In [83]:
ad_recs_annotated[ad_recs_annotated.is_success=='yes'].groupby(['region']).apply(lambda grp: grp.rec_provider.value_counts().head(1)).to_frame() # 16th question

,,rec_provider
region,,
DE,DNNQ,27215
GB,DNNQ,34681
IN,UserPopQ,30407
JP,DNNQ,20113
US,DNNY,25705


#### Q21 - Present a table with a success rate, sever defects rate and CI (for each metric) per rec type and region

In [88]:
import numpy as np
import math
from scipy import stats
def estimate_head_probability(head_prob, n_samples, alpha):
    n_sided = 2
    one_trail = np.random.rand(n_samples) < head_prob
    estimated_head_prob = sum(one_trail) / n_samples
    #standard_error = math.sqrt(head_prob * (1-head_prob) / n_samples)
    estimated_standard_error = math.sqrt(estimated_head_prob * (1-estimated_head_prob) / n_samples)
    z_crit = stats.norm.ppf(1-alpha/n_sided)
    confidance_interval = (estimated_head_prob - z_crit * estimated_standard_error, estimated_head_prob + z_crit * estimated_standard_error)
    return estimated_head_prob, confidance_interval
def estimate_head_probability_for_graph(head_prob, n_samples, alpha):
    n_sided = 2
    one_trail = np.random.rand(n_samples) < head_prob
    estimated_head_prob = sum(one_trail) / n_samples
    estimated_standard_error = math.sqrt(estimated_head_prob * (1-estimated_head_prob) / n_samples)
    z_crit = stats.norm.ppf(1-alpha/n_sided)
    confidance_interval_half_len = z_crit * estimated_standard_error
    return confidance_interval_half_len
alpha = 0.01
estimated_head_prob, confidance_interval = estimate_head_probability(0.6, 100, alpha)

In [89]:
q21_success_rate_df = ad_recs_annotated.groupby(['recommendation_type','region']).new_success_col.mean().to_frame().rename(columns={"new_success_col": "success_rate"})
q21_success_rate_df['success_rate_CI'] = ad_recs_annotated.groupby(['recommendation_type','region']).new_success_col.apply(lambda row: estimate_head_probability(row, row.shape[0], 0.01)[1])
q21_success_rate_df['severe_defects_rate'] = ad_recs_annotated.groupby(['recommendation_type','region']).is_severe_fixed.mean()
q21_success_rate_df['severe_defects_CI'] = ad_recs_annotated.groupby(['recommendation_type','region']).is_severe_fixed.apply(lambda row: estimate_head_probability(row, row.shape[0], 0.01)[1])
q21_success_rate_df

success_rate  \
recommendation_type region                 
Qality              DE          0.806681   
                    GB          0.850367   
                    IN          0.806129   
                    JP          0.777806   
                    US          0.899070   
Yield               DE          0.845986   
                    GB          0.772476   
                    IN          0.719186   
                    JP          0.683988   
                    US          0.828812   

                                                     success_rate_CI  \
recommendation_type region                                             
Qality              DE      (0.8021448961398179, 0.8112162729624786)   
                    GB      (0.8468789499599271, 0.8538544477318154)   
                    IN      (0.8010913201825111, 0.8111675655513576)   
                    JP      (0.7721201340158229, 0.7834918088074875)   
                    US      (0.8952172388739428, 0.9029234658827954)   
Yield               DE      (0.8374558088934787, 0.8545152397515305)   
                    GB      (0.7632099368950809, 0.7817425702840664)   
                    IN      (0.7135576908201734, 0.7248140173732925)   
                    JP      (0.6687276536774429, 0.6992489621069027)   
                    US      (0.8249554392914834, 0.8326686265609805)   

                            severe_defects_rate  \
recommendation_type region                        
Qality              DE                 0.121503   
                    GB                 0.108295   
                    IN                 0.088980   
                    JP                 0.156587   
                    US                 0.074470   
Yield               DE                 0.080121   
                    GB                 0.177675   
                    IN                 0.191504   
                    JP                 0.146639   
                    US                 0.102444   

                                                     severe_defects_CI  
recommendation_type region                                              
Qality              DE      (0.11775065038688445, 0.12525561266113225)  
                    GB      (0.10525664539129584, 0.11133341555707815)  
                    IN      (0.08535129140049501, 0.09260817202846809)  
                    JP      (0.15161704525857425, 0.16155783985236769)  
                    US      (0.07111235316379799, 0.07782853900200973)  
Yield               DE      (0.07370597528937695, 0.08653640814775485)  
                    GB      (0.16922607176053078, 0.18612331340992794)  
                    IN       (0.18657601875230262, 0.1964318768554701)  
                    JP      (0.13502703339577798, 0.15825000460357247)  
                    US      (0.09933953109369845, 0.10554884518803886)

#### Q24 - Present WoW table/graph with success rate and CI (see image below) per region and rec provider

In [92]:
q24_success_rate_df = ad_recs_annotated.groupby(['recommendation_type','region']).new_success_col.mean().to_frame().rename(columns={"new_success_col": "success_rate"})
q24_success_rate_df['success_rate_CI'] = ad_recs_annotated.groupby(['recommendation_type','region']).new_success_col.apply(lambda row: estimate_head_probability_for_graph(row, row.shape[0], 0.01))
q24_success_rate_df['severe_defects_rate'] = ad_recs_annotated.groupby(['recommendation_type','region']).is_severe_fixed.mean()
q24_success_rate_df['severe_defects_CI'] = ad_recs_annotated.groupby(['recommendation_type','region']).is_severe_fixed.apply(lambda row: estimate_head_probability_for_graph(row, row.shape[0], 0.01))
q24_success_rate_df = q24_success_rate_df.unstack(['recommendation_type'])
q24_success_rate_df.reset_index()
q24_success_rate_df.index
q24_success_rate_df
# px.line(q24_success_rate_df, x='region', y='success_rate')

success_rate           success_rate_CI            \
recommendation_type       Qality     Yield          Qality     Yield   
region                                                                 
DE                      0.806681  0.845986        0.004536  0.008530   
GB                      0.850367  0.772476        0.003488  0.009266   
IN                      0.806129  0.719186        0.005038  0.005628   
JP                      0.777806  0.683988        0.005686  0.015261   
US                      0.899070  0.828812        0.003853  0.003857   

                    severe_defects_rate           severe_defects_CI            
recommendation_type              Qality     Yield            Qality     Yield  
region                                                                         
DE                             0.121503  0.080121          0.003752  0.006415  
GB                             0.108295  0.177675          0.003038  0.008449  
IN                             0.088980  0.191504          0.003628  0.004928  
JP                             0.156587  0.146639          0.004970  0.011611  
US                             0.074470  0.102444          0.003358  0.003105